# Snowflake Storage Cost SQL Queries — Versión para **Snowflake Notebooks**
**Fecha:** 2025-11-06
**Autor:** Adolfo Orozco
**Tags:** #snowflake #snowpark #python #sql #cost #performance #recipes

Este cuaderno está optimizado para ejecutarse dentro de **Snowflake Notebooks** usando **Snowpark**. 
Cuando se ejecute fuera de Snowflake Notebooks (por ejemplo, en tu Jupyter local), hará *fallback* a una conexión estándar con credenciales.


El **COSTO TOTAL** por ALMACENAMIENTO (storage) es la **suma** de los costos asociados con:

- Staged file storage
- Database table storage
- Fail-safe and Time Travel storage

Snowflake cobra el almacenamiento por **Terabyte al mes** (TB/month) y esta métrica depende de la región de nube en la que se encuentre desplegada nuestra instancia de Snowflake dado que para cada región el TB/month puede tener un precio diferente.  Por ejemplo, para una instancia de Snowflake desplegada en Amazon Web Services (AWS) en la región de US-EAST se tiene que el precio del TB/month sería de 23 USD de acuerdo a la información publicada en el sitio oficial de Snowflake en el siguiente enlance https://www.snowflake.com/en/pricing-options/.

## 1. Requisitos previos

- En **Snowflake Notebooks** no necesitas credenciales: se usa `get_active_session()`.
- Si corres localmente:
  - Paquetes: `snowflake-snowpark-python[pandas]`, `python-dotenv` (opcional).
  - Credenciales de Snowflake (account, user, password/SSO/TOTP, role, warehouse, database, schema).


In [ ]:
# 1.1 (Opcional, ejecución local) Instalar dependencias
# %pip install --quiet "snowflake-snowpark-python[pandas]" python-dotenv


## 2. Conexión con Snowflake Notebooks (Snowpark)

- En Snowflake Notebooks: `get_active_session()` devuelve una sesión lista para usar.
- En local: `Session.builder.configs({...}).create()` con tus credenciales.


In [ ]:
# 2.1 Crear/obtener la sesión de Snowpark
from __future__ import annotations

import os
import getpass

try:
    # Disponible dentro de Snowflake Notebooks
    from snowflake.snowpark.context import get_active_session
    from snowflake.snowpark import Session
    session = get_active_session()
    IN_SNOWFLAKE_NOTEBOOK = True
except Exception:
    # Fallback local
    from snowflake.snowpark import Session
    IN_SNOWFLAKE_NOTEBOOK = False

def get_session() -> "Session":
    """Devuelve una sesión de Snowpark. Usa la sesión activa si estamos en Snowflake Notebooks.
    Si se ejecuta localmente, crea una sesión con credenciales proporcionadas por variables de entorno o input seguro."""
    global session
    if IN_SNOWFLAKE_NOTEBOOK:
        return session
    
    # Lectura de credenciales para ejecución local
    connection_parameters = {
        "account":   os.getenv("SNOWFLAKE_ACCOUNT", "your_account"),
        "user":      os.getenv("SNOWFLAKE_USER", "your_user"),
        "password":  os.getenv("SNOWFLAKE_PASSWORD") or getpass.getpass("Snowflake password: "),
        "role":      os.getenv("SNOWFLAKE_ROLE", "ACCOUNTADMIN"),
        "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH"),
        "database":  os.getenv("SNOWFLAKE_DATABASE", "SNOWFLAKE"),
        "schema":    os.getenv("SNOWFLAKE_SCHEMA", "ACCOUNT_USAGE"),
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

# Probar conexión
_sess = get_session()
_sess.sql("SELECT CURRENT_VERSION() AS VERSION").to_pandas()


## 3. Utilidades y contexto de sesión
Incluimos un *helper* para ejecutar SQL con Snowpark y devolver `pandas.DataFrame`, 
junto con una consulta para inspeccionar el contexto actual.


In [ ]:
# 3.1 Helper para ejecutar SQL y devolver pandas DataFrame
import pandas as pd

def run_sql_df(sql: str) -> pd.DataFrame:
    s = get_session()
    return s.sql(sql).to_pandas()

# 3.2 Contexto actual
ctx_sql = '''
SELECT CURRENT_ROLE() AS ROLE,
       CURRENT_WAREHOUSE() AS WAREHOUSE,
       CURRENT_DATABASE() AS DATABASE,
       CURRENT_SCHEMA() AS SCHEMA;
'''
run_sql_df(ctx_sql)


## 4. STAGED FILES: volúmen de almacenamiento (en Terabytes) de objetos en stages (por año por mes)

**Propósito:** Calcular el volumen en Terabytes de todos los objetos almacenados en stages por año por mes.


In [ ]:
# 4.1 Carga diaria vs. colas (últimos 30 días)
sql_load_vs_queue = '''
WITH stage_usage AS (
  SELECT
      USAGE_DATE,
      AVERAGE_STAGE_BYTES as storage_bytes
  FROM SNOWFLAKE.ACCOUNT_USAGE.STAGE_STORAGE_USAGE_HISTORY
)
SELECT
    YEAR(usage_date)                                   AS usage_year,
    MONTH(usage_date)                                  AS usage_month,
    TO_VARCHAR(DATE_TRUNC('month', usage_date), 'YYYY-MM') AS month_key,
    -- Sum all stage storage for the month and convert bytes to TiB
    ROUND(SUM(storage_bytes) / POWER(1024, 4), 4)      AS total_stage_volume_tib
FROM stage_usage
GROUP BY 1, 2, 3
ORDER BY usage_year, usage_month;
'''
run_sql_df(sql_load_vs_queue)


## 5. Volúmen de almacenamiento de objetos de tipo TABLA en Terabytes (en los últimos 12 meses)

**Propósito:** Calcular el volumen en Terabytes de todos los objetos de tipo tabla en la cuenta de Snowflake en los últimos 12 meses (incluyendo time-travel y fail-safe).


In [ ]:
# 5.1 Carga diaria vs. colas (últimos 30 días)
sql_load_vs_queue = '''
WITH daily AS (
  SELECT
      usage_date,
      average_database_bytes,
      average_failsafe_bytes
  FROM SNOWFLAKE.ACCOUNT_USAGE.DATABASE_STORAGE_USAGE_HISTORY
  WHERE usage_date BETWEEN DATEADD(month, -12, CURRENT_DATE()) AND CURRENT_DATE()
)
SELECT
    YEAR(usage_date)                                        AS usage_year,
    MONTH(usage_date)                                       AS usage_month,
    TO_VARCHAR(DATE_TRUNC('month', usage_date), 'YYYY-MM')  AS month_key,

    -- Monthly averages (across the days in that month), converted to TiB
    ROUND(AVG(average_database_bytes) / POWER(1024, 4), 4)  AS avg_table_storage_tib,
    ROUND(AVG(average_failsafe_bytes) / POWER(1024, 4), 4)  AS avg_failsafe_storage_tib,
    ROUND(
      AVG(average_database_bytes + average_failsafe_bytes) / POWER(1024, 4), 4
    ) AS avg_total_storage_tib

FROM daily
GROUP BY 1, 2, 3
ORDER BY usage_year, usage_month;
'''
run_sql_df(sql_load_vs_queue)
